In [30]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import time
import pyautogui
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import os 


In [31]:
base_url = "https://www.grailed.com/designers/john-elliott"

# open up chrome. Don't use headless otherwise the feed-item doesn't fully load all the way
# headless = dont open up the chrome
#chrome_options.add_argument('--headless')
driver = webdriver.Chrome()
driver.get(base_url)

# wait 30 sec
timeout = 30

In [32]:
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='feed-item']")))
except TimeoutException:
    print("Timed out waiting for page to load")
    driver.quit()


#Don't want the items in staff recommendations and other items, just want the ones that show up in the search
results = driver.find_elements(By.XPATH, '//div[@class="FiltersInstantSearch"]//div[@class="feed-item"]')

# just so i can see how many iterations i probably need. Should be 40
print(len(results))


pyautogui.keyDown('ctrl')
pyautogui.keyDown('shift')
pyautogui.press('c')
pyautogui.keyUp('ctrl')
pyautogui.keyUp('shift')

try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.CLASS_NAME, '-listing-count')))
except TimeoutException:
    print("Timed out waiting for page to load")
    driver.quit()

# Get the number of listings and turn it into a number to find out how many pages I need to scroll
ListingNumber = driver.find_element(By.XPATH, '//div[@class="-listing-count"]//span').text

ListingNumber=int(ListingNumber.split(" ")[0])

pyautogui.keyDown('ctrl')
pyautogui.keyDown('shift')
pyautogui.press('i')
pyautogui.keyUp('ctrl')
pyautogui.keyUp('shift')

# Number of Scrolls, add one in case the function rounds down
ScrollNumber=round(ListingNumber/len(results))+1
print(ScrollNumber)

# Start the scroll
for i in range(0,ScrollNumber):
    #results = driver.find_elements_by_xpath('//div[@class="feed-item"]')
    #Results=Results+results
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)

# Run again to see total number of results the scraper is going to get. Should be the same as ListingNumber above
results = driver.find_elements(By.CLASS_NAME, 'feed-item')
print(len(results))
print(ListingNumber)

40
100
1003
3968


In [33]:
dataList = []
# Start scraping. Notice there's a '.' in front of //div. It's so that you start searching in each feed-item post
# Certain items don't have some of these tags in them, so make a "try-except" statement 
for result in results:

    itemDataList = []
    try:
        name = result.find_element(By.XPATH, './/p[@data-cy="listing-title"]').text
        itemDataList.append(name)
    except NoSuchElementException:
        itemDataList.append("")

    try:
        new_price = result.find_element(By.XPATH, './/span[@data-testid="Current"]').text
        old_price = result.find_element(By.XPATH, './/span[@data-testid="Original"]').text
        itemDataList.append("")
        itemDataList.append(new_price)
        itemDataList.append(old_price)
        
    except NoSuchElementException:
        try:
            price = result.find_element(By.XPATH, './/span[@data-testid="Current"]').text
            itemDataList.append(price)
            itemDataList.append("")
            itemDataList.append("")
        except NoSuchElementException:
            itemDataList.append("")
            itemDataList.append("")
            itemDataList.append("")
    try:
        size = result.find_element(By.XPATH, './/p[@class="ListingMetadata_size__Ho8j4"]').text
        itemDataList.append(size)
    except NoSuchElementException:
        itemDataList.append("")

    try:
        curr_time=result.find_element(By.XPATH, ".//span[@class='ListingAge_dateAgo__JEsJV']").text
        itemDataList.append(curr_time)
    except NoSuchElementException:
        itemDataList.append("")
    try:
        last_bump=result.find_element(By.XPATH, ".//span[@class='ListingAge_strikeThrough__rrCHR']").text
        itemDataList.append(last_bump)
    except NoSuchElementException:
        itemDataList.append("")
    try:
        link = result.find_element(By.XPATH, './a').get_attribute("href")
        itemDataList.append(link)
    except NoSuchElementException:
        itemDataList.append("")
    dataList.append(itemDataList)
# Turn the lists into a DataFrame with some nice column names 
ItemDF=pd.DataFrame(dataList,columns=['Name','Price','NewPrice','OldPrice','Size','Time','LastBump','Link'])
print(ItemDF.head())

                                                Name Price NewPrice OldPrice  \
0                                LEATHER DOWN JACKET  $899                     
1                   John Elliot Corduroy Ball Shorts           $135     $150   
2         John Elliot Reversible Nylon Fleece Jacket  $250                     
3  John Elliot Reversible Fleece Nylon Color Bloc...  $250                     
4                John Elliott Leather Thumper Jacket  $750                     

  Size                Time  LastBump  \
0    L           1 day ago             
1   32  2 days ago(2 days)  (2 days)   
2   XL          3 days ago             
3   XL          3 days ago             
4    L   about 3 hours ago             

                                                Link  
0  https://www.grailed.com/listings/59057179-john...  
1  https://www.grailed.com/listings/58987422-john...  
2  https://www.grailed.com/listings/58975832-john...  
3  https://www.grailed.com/listings/58975890-john...  
4  

In [ ]:
chrome_options = webdriver.ChromeOptions()
# headless:dont open up the chrome
# chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.grailed.com/designers/chrome-hearts")
# timeout after 30 sec
timeout = 30
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='feed-item']")))
except TimeoutException:
    print("Timed out waiting for page to load")
    driver.quit()


# Lists to append everything to
metaDataList = []
### Start Process. Link is a list of links from ItemDF
for link in ItemDF['Link']:

    driver.get(link)
    time.sleep(5)
    # itemData = []

    # try:
    #     sold=driver.find_element(By.XPATH, '//a[@class="-link"]/span[2]').text
    #     itemData.append(sold)
    # except NoSuchElementException:
    #     itemData.append("")

    # try:
    #     description=driver.find_element(By.XPATH, '//div[@class="listing-description"]').text
    #     itemData.append(description)
    # except NoSuchElementException:
    #     itemData.append(description)

    # try:
    #     fullsize=driver.find_element(By.XPATH, '//h2[@class="listing-size sub-title"]').text
    #     itemData.append(fullsize)
    # except NoSuchElementException:
    #     itemData.append("")

    # try:
    #     postedtime=driver.find_element(By.XPATH, '//div[@class="-metadata"]/span[2]').text
    #     itemData.append(postedtime)
    # except NoSuchElementException:
    #     itemData.append("")

    # try:
    #     bumpedtime=driver.find_element(By.XPATH, '//div[@class="-metadata"]/span[4]').text
    #     itemData.append(bumpedtime)
    # except NoSuchElementException:
    #     itemData.append("")

    # try:
    #     location=driver.find_element(By.XPATH, '//label[@class="--label"]').text
    #     itemData.append(location)
    # except NoSuchElementException:
    #     itemData.append("")
    
    images = driver.find_elements(By.TAG_NAME, 'img')
    print(images)
    imageList = []
    for image in images:
        imageList.append(image.get_attribute("src"))
    # itemData.append(count_of_divs)
    # metaDataList.append(itemData)
    print(imageList)


# Turn it into DataFrame
# dataDF=pd.DataFrame(metaDataList,
# columns=['UserName','Sold','FeedBack','CurrentListings','Description','ProfileLink','FeedbackLink','FollowerCount','FullSize','PostedTime','BumpedTime','Location','PictureNumber'])
# dataDF

Timed out waiting for page to load


KeyboardInterrupt: 

In [42]:
chrome_options = webdriver.ChromeOptions()
# headless:dont open up the chrome
# chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
# timeout after 30 sec
timeout = 30



# Lists to append everything to
metaDataList = []
### Start Process. Link is a list of links from ItemDF
for link in ItemDF['Link']:
    print(link)
    driver.get(link)

    images = driver.find_elements(By.TAG_NAME, 'img')
    print(images)
    imageList = []
    i = 0
    for image in images:
        image_link = image.get_attribute("src")
        imageList.append(image_link)
        driver.get(image_link)
        image = driver.find_element(By.TAG_NAME, 'img')
        print(image)
        image.screenshot(f'/Images/Clothes{i}.png')  
        driver.get(link)
        i+=1
        break
    print(imageList)
    break

https://www.grailed.com/listings/59057179-john-elliott-leather-down-jacket?g_aidx=Listing_by_heat_production&g_aqid=52382892a683745a17eea41f373f14e7
[<selenium.webdriver.remote.webelement.WebElement (session="70b519775856c52ca02ddf360894a6ad", element="f.CAB0132C6F74DB96100E04EF555C9A45.d.5291A130875BD31882B389DC94690660.e.58")>, <selenium.webdriver.remote.webelement.WebElement (session="70b519775856c52ca02ddf360894a6ad", element="f.CAB0132C6F74DB96100E04EF555C9A45.d.5291A130875BD31882B389DC94690660.e.62")>, <selenium.webdriver.remote.webelement.WebElement (session="70b519775856c52ca02ddf360894a6ad", element="f.CAB0132C6F74DB96100E04EF555C9A45.d.5291A130875BD31882B389DC94690660.e.84")>, <selenium.webdriver.remote.webelement.WebElement (session="70b519775856c52ca02ddf360894a6ad", element="f.CAB0132C6F74DB96100E04EF555C9A45.d.5291A130875BD31882B389DC94690660.e.88")>, <selenium.webdriver.remote.webelement.WebElement (session="70b519775856c52ca02ddf360894a6ad", element="f.CAB0132C6F74DB961

In [ ]:
def photo_downloader_displayer(folder_name, df):
    
    
    # create folder
    
    directory = f'../{folder_name}_pics/'
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    
    # Displays images and necessary information in the jupyter notebook
    for idx in df.index:
        display(Image(f"{directory}{str(idx)}_screenshot.png"))
        print(f"Name: {df.at[idx, 'name']}")
        print(f"Price: ${df.at[idx, 'price_dollar']}")
        print(f"URL: {df.at[idx, 'url']}")
        print('')
    
    return df





In [26]:
link = "https://media-assets.grailed.com/prd/listing/temp/a16aad8200bc4edfbce680d3e0795d4d?"
folder_name = "Image"
directory = f'../{folder_name}_pics/'
try:
    if not os.path.exists(directory):
        os.makedirs(directory)
except OSError:
    print ('Error: Creating directory. ' +  directory)

